In [ ]:
## Example notebook for running CellStitch

In [13]:
import os
import time

import numpy as np
import torch
import tifffile
from cellpose.models import Cellpose
from skimage import io

from cellstitch.pipeline import full_stitch

In [2]:
# Plotting specifications
from matplotlib import rcParams
from IPython.display import display
rcParams.update({'font.size': 10})

### (1). Load example pairs of raw image & ground-truth mask

In [4]:
# Fill in filename for raw image (including the paths)
filename = "/home/clement/Images/zebra_sketchpose/20240513_OmpYFP_84mKate_O40_05u_80u_12ss_Position3_2_cmle_ch00.tif"

# Fill in on the path you would like to store the stitched mask
output_path = "/home/clement/Images/zebra_sketchpose/"
output_filename = '20240513_OmpYFP_84mKate_O40_05u_80u_12ss_Position3_2_cmle_ch00_seg.tif'

Example code snippet
```python
filename = '../data/plant_atlas/Anther/Anther_20.tif'
maskname = '../data/plant_atlas/Anther_masks/Anther_20.tif'

output_path = '../results/'
if not os.exist(output_path):
    os.makedirs(output_path, exist_ok=True)
    
output_filename = 'cellstitch_sample_pred.npy'
```

In [5]:
# Load image & masks
if filename[-3:] == 'npy':  # image in .npy format
    img = np.load(filename)
elif filename[-3:] == 'tif': # imagge in TIFF format
    img = tifffile.imread(filename)
else:
    try:
        img = io.imread(filename)
    except:
        raise IOError('Failed to load image {}'.format(filename))

"""if maskname[-3:] == 'npy':  # image in .npy format
    mask = np.load(maskname)
elif filename[-3:] == 'tif': # imagge in TIFF format
    mask = tifffile.imread(maskname)
else:
    try:
        mask = io.imread(maskname)
    except:
        raise IOError('Failed to load image {}'.format(filename))"""


"if maskname[-3:] == 'npy':  # image in .npy format\n    mask = np.load(maskname)\nelif filename[-3:] == 'tif': # imagge in TIFF format\n    mask = tifffile.imread(maskname)\nelse:\n    try:\n        mask = io.imread(maskname)\n    except:\n        raise IOError('Failed to load image {}'.format(filename))"

### (2). Define configs & parameters

In [9]:
# load cellpose model for backbone segmentation
# you can also replace with any 2D segmentation model that works the best for your dataset
flow_threshold = 0.4
use_gpu = True if torch.cuda.is_available() else False
if use_gpu is True:
    print("Using GPU")
model = Cellpose(model_type='cyto3', gpu=use_gpu)


Using GPU


### (3). Run CellStitch

In [ ]:
tic = time.time()
xy_masks, _, _, _ = model.eval(list(img), flow_threshold=flow_threshold, channels = [0,0], diameter=50)
xy_masks = np.array(xy_masks)
print("xy masks computed")

yz_masks, _, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold, channels = [0,0], diameter=30)
yz_masks = np.array(yz_masks).transpose(1,0,2)
print("yz masks computed")

xz_masks, _, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold, channels = [0,0], diameter=30)
xz_masks = np.array(xz_masks).transpose(2,1,0)
print("xz masks computed")


cellstitch_masks = full_stitch(xy_masks, yz_masks, xz_masks)

computation_time = time.time() - tic
print("Computation time: {}".format(computation_time))

### (4). Save the Stitching results:

In [19]:
io.imsave(os.path.join(output_path, output_filename), cellstitch_masks)
print(cellstitch_masks.shape)


/tmp/ipykernel_110555/2495679190.py:1: UserWarning: /home/clement/Images/zebra_sketchpose/20240513_OmpYFP_84mKate_O40_05u_80u_12ss_Position3_2_cmle_ch00_seg.tif is a low contrast image
  io.imsave(os.path.join(output_path, output_filename), cellstitch_masks)


(181, 1433, 1003)


In [100]:
import pygame
from gtts import gTTS

# Définir la phrase à dire en français
phrase = ("La segmentation est terminée")

# Utiliser gTTS pour générer le fichier audio en français
tts = gTTS(text=phrase, lang='en-au')

# Sauvegarder le fichier audio
tts.save("segmentation_terminee.mp3")

# Initialiser Pygame
pygame.init()

# Charger le fichier audio
pygame.mixer.music.load("segmentation_terminee.mp3")

# Jouer le fichier audio
pygame.mixer.music.play()

# Attendre que la lecture soit terminée
while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)  # Limiter la fréquence de rafraîchissement

# Quitter Pygame
pygame.quit()
